<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

In [2]:
# Parameters
input_geojson_path = "training-data/postcards/tulagi_postcard.geojson"
output_csv_path = "automated_notebook_runs/sampled_data_csvs/tulagi_postcard_sampled_data.csv"


## Step 1.1: Configure the environment

In [3]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [4]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [5]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "tulagi"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-tulagi-27072025


In [6]:
gdfs = []
postcards_path = "training-data/postcards"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [7]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-efate-27072025_postcard.geojson
suva_west_postcard.geojson
paunganisu_postcard.geojson
mana-north_postcard.geojson
tulagi_postcard.geojson
serua_2_postcard.geojson
suva_central_postcard.geojson
efate_postcard.geojson
suva_east_postcard.geojson
bootless-bay_postcard.geojson
serua_postcard.geojson
korotogo_postcard.geojson
yanuca_postcard.geojson
mana-south_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [8]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [9]:
# filename = "nm-efate-27072025_postcard.geojson"

In [10]:
training = gpd.read_file(f"training-data/postcards/{site}_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [11]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 2 items in for 2024


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 22MB
Dimensions:      (y: 784, x: 762, time: 1)
Coordinates:
  * y            (y) float64 6kB -1.009e+06 -1.009e+06 ... -1.017e+06 -1.017e+06
  * x            (x) float64 6kB 1.124e+06 1.124e+06 ... 1.132e+06 1.132e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 1MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    red          (time, y, x) uint16 1MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    blue         (time, y, x) uint16 1MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    green        (time, y, x) uint16 1MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    emad         (time, y, x) float32 2MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    smad         (time, y, x) float32 2MB dask.array<chunksize=(1, 784, 762), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 1MB dask.array<chunksize=

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33697 instead
  warnings.warn(


In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled

<xarray.Dataset> Size: 65MB
Dimensions:      (y: 784, x: 762)
Coordinates:
  * y            (y) float64 6kB -1.009e+06 -1.009e+06 ... -1.017e+06 -1.017e+06
  * x            (x) float64 6kB 1.124e+06 1.124e+06 ... 1.132e+06 1.132e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 5MB 0.4332 0.4514 0.4192 ... 0.0218 0.0218
    red          (y, x) float64 5MB 0.0404 0.0407 0.0389 ... 0.0248 0.0254
    blue         (y, x) float64 5MB 0.0486 0.0501 0.0475 ... 0.0441 0.0443
    green        (y, x) float64 5MB 0.0759 0.0799 0.0762 ... 0.0322 0.0324
    emad         (y, x) float32 2MB 0.1689 0.1618 0.1425 ... 0.05319 0.05392
    smad         (y, x) float32 2MB 1.093e-07 1.11e-07 ... 3.812e-06 3.739e-06
    ...           ...
    swir16       (y, x) float64 5MB 0.1788 0.1789 0.1808 ... 0.0212 0.0216
    swir22       (y, x) float64 5MB 0.078 0.0778 0.079 ... 0.0194 0.0192 0.0195
    coastal      (y, x) float64 5MB 0.0458 0.0454 0.0493 ... 0.0516 0.0517
    rededge1     (y, x) float64 5MB 0.1209 0.1208 0.1207 ... 0.0261 0.0265
    rededge2     (y, x) float64 5MB 0.3509 0.3518 0.3424 ... 0.0243 0.0246
    rededge3     (y, x) float64 5MB 0.4499 0.4509 0.4409 ... 0.0254 0.0248 0.025

In [17]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [18]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [19]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [20]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 139MB
Dimensions:      (y: 784, x: 762)
Coordinates:
  * y            (y) float64 6kB -1.009e+06 -1.009e+06 ... -1.017e+06 -1.017e+06
  * x            (x) float64 6kB 1.124e+06 1.124e+06 ... 1.132e+06 1.132e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 5MB 0.4332 0.4514 0.4192 ... 0.0218 0.0218
    red          (y, x) float64 5MB 0.0404 0.0407 0.0389 ... 0.0248 0.0254
    blue         (y, x) float64 5MB 0.0486 0.0501 0.0475 ... 0.0441 0.0443
    green        (y, x) float64 5MB 0.0759 0.0799 0.0762 ... 0.0322 0.0324
    emad         (y, x) float32 2MB 0.1689 0.1618 0.1425 ... 0.05319 0.05392
    smad         (y, x) float32 2MB 1.093e-07 1.11e-07 ... 3.812e-06 3.739e-06
    ...           ...
    homogeneity  (y, x) float32 2MB 0.9792 0.9861 1.0 0.9931 ... nan nan nan nan
    energy       (y, x) float32 2MB 0.9314 0.9724 1.0 0.9862 ... nan nan nan nan
    ASM          (y, x) float32 2MB 0.8676 0.9456 1.0 0.9725 ... nan nan nan nan
    correlation  (y, x) float32 2MB 0.5495 -0.01408 1.0 ... nan nan nan
    mean         (y, x) float32 2MB 1.951 1.986 2.0 1.993 ... nan nan nan nan
    entropy      (y, x) float32 2MB 0.4729 0.2109 -1.443e-10 ... nan nan nan

In [21]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [22]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [23]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1010155.0,1125625.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1010215.0,1125625.0,3832,2024-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1010465.0,1125795.0,3832,2024-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1010165.0,1126205.0,3832,2024-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1010205.0,1126225.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1012935.0,1130485.0,3832,2024-01-01
610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1013145.0,1130595.0,3832,2024-01-01
611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1013335.0,1130585.0,3832,2024-01-01


In [24]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
6,9,0.2434,0.1300,0.0994,0.1272,0.141290,6.065071e-07,0.000011,0.2123,0.1977,...,0.884722,0.646990,0.418596,0.865188,1.354167,2.010009,-1010245.0,1126345.0,3832,2024-01-01
11,9,0.1340,0.0433,0.0401,0.0507,0.062188,3.666937e-07,0.000010,0.1421,0.1589,...,0.841095,0.596576,0.355903,0.405610,0.937500,2.309322,-1010375.0,1126425.0,3832,2024-01-01
12,9,0.1957,0.0436,0.0403,0.0546,0.069990,4.105151e-07,0.000008,0.2055,0.1605,...,0.825000,0.488684,0.238812,0.883978,1.333333,2.946210,-1010395.0,1126435.0,3832,2024-01-01
14,9,0.1025,0.1140,0.1097,0.1392,0.197182,3.384769e-06,0.000024,0.1122,0.1695,...,0.829984,0.433347,0.187789,0.891762,6.451389,3.363655,-1010455.0,1126395.0,3832,2024-01-01
15,9,0.2627,0.0308,0.0338,0.0536,0.059826,8.151531e-08,0.000004,0.2837,0.3013,...,0.861111,0.399653,0.159722,0.974230,4.208333,3.473872,-1010445.0,1126455.0,3832,2024-01-01


In [25]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [26]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [27]:
training_array=training_array[standard_schema]

In [28]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [29]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [30]:
training_array["cc_id"].dtype

dtype('int32')